# Extract images from a live video to create a dataset

We now use a simple GUI to generate new dataset for image segmentation. 

Here are the steps:


* Get an image from a camera (could later be a video or images). 
* Click on the video to extract one image, and then click again to label the oject on the image.
* Save image, coordinate and mask to the dataset.
* Split images and mask as a train and validation dataset.

You want at least 150 imgaes to get ok tracking. We can further improve the model later on.

In [11]:
%matplotlib widget
import matplotlib.pyplot as plt
import cv2
import numpy as np
import ipywidgets as widgets

from unetTracker.trackingProject import TrackingProject
from unetTracker.dataset import UNetDataset
from unetTracker.camera import USBCamera, bgr8_to_jpeg
from unetTracker.unetGUI import LabelFromCameraGUI



## Load a TrackingProject object from file



In [2]:
project = TrackingProject(name="faceTrack",root_folder = "/home/kevin/Documents/trackingProjects/")

Project directory: /home/kevin/Documents/trackingProjects/faceTrack
Loading /home/kevin/Documents/trackingProjects/faceTrack/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_size': [480, 640], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'faceTrack', 'normalization_values': {'means': [0.5110162496566772, 0.4608974754810333, 0.4772901237010956], 'stds': [0.2727729380130768, 0.2578601539134979, 0.256255567073822]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (128.0, 0.0, 128.0)], 'objects': ['nose', 'chin', 'rEye', 'lEye'], 'target_radius': 10}


In [3]:
project.mask_dir

'/home/kevin/Documents/trackingProjects/faceTrack/dataset/masks'

## Dataset object

A Dataset object is created to save labeled images, masks and coordinates. It is also used to load the data into memory when training your model. This is a class inherited from `torch.utils.data.Dataset`.


In [4]:
dataset = UNetDataset(image_dir=project.image_dir, mask_dir=project.mask_dir, coordinate_dir=project.coordinate_dir)

You can know how many images are in your dataset.

In [5]:
len(dataset)

717

You can also load any of the images in your dataset, together with mask and coordinates.

In [6]:
image,mask,coordinates = dataset[0]

In this case we are getting an image, mask, and coordinates of objects.

The image and mask are `torch.tensor` instead of `np.array`

## Camera class


In [7]:
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 1 Dez 10 10:01 /dev/video1
crw-rw----+ 1 root video 81, 0 Dez 10 10:01 /dev/video0


In [8]:
camera = USBCamera(width=project.image_size[1], height=project.image_size[0])

In [9]:
image = camera.read()
print(image.shape)


(480, 640, 3)


In [12]:
imgWidget = widgets.Image(format='jpeg')
imgWidget.value = bgr8_to_jpeg(image)
display(imgWidget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Start the GUI to extract and label frames from a camera 

1. Click on the top left image to capture a frame from the camera feed. A larger image will appear below the entry boxes for the coordinates. 
2. Your body parts are listed above the large image. In the large image, click on the object that is selected in the radio button. You should see a label apper in the picture at the very bottom.
3. Repeat for all the body parts visible on the image. If you leave the coordinate of an object at 0,0, it is considered not in the image.
4. Once all the body parts are labeled, click on `Save labelled frame`.
5. Go back to step 1 and repeat.

In [13]:
LabelFromCameraGUI(camera,project,dataset)

LabelFromCameraGUI(children=(HBox(children=(Image(value=b'', format='jpeg', height='240.0', width='320.0'), Im…

In [15]:
print(len(dataset))


720


When you click on `Save labelled frame`, a new data point is added to your dataset. This is stored in a subdirectory of your project directory called `dataset`. There are 3 directories there: images, masks and coordinates. This data will be used to train the network and assess tracking quality. 